# Clustering algorithms

In [115]:
using Random
using StatsBase
using Plots
using Statistics
using LinearAlgebra

In [2]:
function InitLattice(N)
    #Lattice = ones(Int8, N) # all site is up
    #Lattice = -1 * ones(Int8, N) # all site is down
    Lattice = sample([-1, 1], Weights([1, 1]), N) # all site random
    
    return Lattice
end

InitLattice (generic function with 1 method)

In [3]:
function PeriodicBoundaryCondition(row, col)
    # All site has 3 of degrees
    adj = zeros(Int64, (row*col, 3))
    
    for i in 1:row
        for j in 1:col
            idx = (i-1)*col + j
            #Horizontal
            adj[idx, 1] = (i-1)*col + ((j + col) % col + 1)
            adj[idx, 2] = (i-1)*col + ((j + col - 2) % col + 1)
            #Vertical
            if i==1
                tmp3 = col + j
                adj[idx, 3] = col + j
            end
            if i==2
                tmp3 = j
                adj[idx, 3] = j
            end
        end
    end                
    return adj
end             

PeriodicBoundaryCondition (generic function with 1 method)

In [11]:
function BootStrap(data, nSample, nBins, T, N)
    mList = Vector{Float64}()
    chiList = Vector{Float64}()
    
    for _ in 1:nSample
        samples = sample(data, nBins, replace=false)
        mag = mean(samples)
        chi = N * (mean(samples .* samples) - mag*mag)/T # .* is elementwise product
        
        append!(mList, mag)
        append!(chiList, chi)
    end
    
    m = mean(mList)
    chi = mean(chiList)
    return m, chi
end    

BootStrap (generic function with 2 methods)

In [5]:
function BFS_Swendsen(Lattice, adj, N, T, J)
    Queue = zeros(Int64, N+1) # Queue for BFS
    Cluster = zeros(Int64, N) # Cluster indexing of each nodes. 0 Value means that the node is not visited
    ClusterID = 0
    for idx in 1:N # search for all nodes
        if Cluster[idx] == 0 # If start node is not visited
            read = 1 # read pointer initialize
            write = 1 # write pointer initialize
            ClusterID+=1 # next cluster ID
            
            ndi = idx # start searching
            Queue[write] = ndi # stack queue
            write+=1 # move write pointer
            Cluster[ndi] = ClusterID # regist node into cluster
            
            while(read != write) # until queue is empty
                read+=1 # move read pointer
                for degree in 1:3 # all node has 3 of degrees
                    ndj = adj[ndi, degree] # choose one of neighbors
                    if Cluster[ndj] == 0 # If the neighbor node is not visited
                        if Lattice[ndi] == Lattice[ndj] # If two nodes have same spin
                            if rand(Float64) < (1 - exp(-2 * J/T)) # With acceptance ratio
                                Cluster[ndj] = ClusterID
                                Queue[write] = ndj
                                write+=1
                            end
                        end
                    end
                end
                ndi = Queue[read]
            end
            
        end
    end
    
    return Cluster
end 

BFS_Swendsen (generic function with 1 method)

In [34]:
function BFS_Wolf(Lattice, adj, N, T, J)
    Queue = zeros(Int64, N+1) # Queue for BFS
    Visited = zeros(Int64, N) # Visited nodes. 0 Value means that the node is not visited
    Cluster = Vector{Int64}() # Set of nodes which belong to cluster
    idx = rand(1:N)

    read = 1 # read pointer initialize
    write = 1 # write pointer initialize

    ndi = idx # start searching
    Queue[write] = ndi # stack queue
    write+=1 # move write pointer
    Visited[ndi] = 1 # regist node into visited
    append!(Cluster, ndi)

    while(read != write) # until queue is empty
        read+=1 # move read pointer
        for degree in 1:3 # all node has 3 of degrees
            ndj = adj[ndi, degree] # choose one of neighbors
            if Visited[ndj] == 0 # If the neighbor node is not visited
                if Lattice[ndi] == Lattice[ndj] # If two nodes have same spin
                    if rand(Float64) < (1 - exp(-2 * J/T)) # With acceptance ratio
                        Visited[ndj] = 1
                        append!(Cluster, ndj)
                        Queue[write] = ndj
                        write+=1
                    end
                end
            end
        end
        ndi = Queue[read]
    end
    
    return Cluster
end 

BFS_Wolf (generic function with 1 method)

In [7]:
function Swendsen(Lattice, adj, N, T, J)
    Cluster = BFS_Swendsen(Lattice, adj, N, T, J)
    nCluster = maximum(Cluster)
    newSpin = sample([-1, 1], Weights([1, 1]), nCluster)
    for idx in 1:N
        Lattice[idx] = newSpin[Cluster[idx]]
    end
end

Swendsen (generic function with 1 method)

In [67]:
function Wolf(Lattice, adj, N, T, J)
    Cluster = BFS_Wolf(Lattice, adj, N, T, J)
    for idx in Cluster
        Lattice[idx] *= -1
    end
end

Wolf (generic function with 1 method)

In [191]:
function CorrelationFunction(Lattice)
    N = length(Lattice)
    L = N÷2
    corLenList = Vector{Float64}()
    term2 = mean(Lattice)
    Ladder = reshape(Lattice, L, 2)
    
    tmp1 = Ladder[:, 1]
    tmp2 = Ladder[:, 2]    
    for j in 1:L÷2
        push!(tmp1, popfirst!(tmp1))
        push!(tmp2, popfirst!(tmp2))
        shiftLattice = vcat(tmp1, tmp2)
        term1 = mean(Lattice .* shiftLattice)
        val = term1 - term2*term2
        append!(corLenList, val)
    end 
    return corLenList
end

CorrelationFunction (generic function with 1 method)

In [228]:
function sampleSwendsen(L, MCstep, adj, T, J, nSample)
    for sample in 1:nSample
        N = L*2
        Lattice = InitLattice(N)
        mStep = zeros(Float64, MCstep)
        for step in 1:MCstep
            Swendsen(Lattice, adj, N, T, J)
            mStep[step] = abs(sum(Lattice)/N)
        end
        mpath = "./data/raw/Swendsen_L$(L)_T$(T)_J$(J)_$(sample).dat"

        open(mpath, "w") do f
            for value in mStep
                println(f, value)
            end
        end

        corLenList = CorrelationFunction(Lattice)
        corpath = "./data/correlation_length/Swendsen_L$(L)_T$(T)_J$(J)_$(sample).dat"

        open(corpath, "w") do f
            for value in corLenList
                println(f, value)
            end
        end
    end
    println("[L$(L), T$(T), J$(J), TotalSample$(nSample)]")
end

sampleSwendsen (generic function with 2 methods)

In [229]:
function IsingSwendsen(LList, TList, JList, MCstep, nSample)
    mSet = Vector{Float64}()
    chiSet = Vector{Float64}()
    
    for J in JList
        for L in LList
            adj = PeriodicBoundaryCondition(2, L)        

            for T in TList
                @time sampleSwendsen(L, MCstep, adj, T, J, nSample)
            end

        end
    end
end

IsingSwendsen (generic function with 3 methods)

In [230]:
function IsingWolf(LList, TList, MCstep)
    mSet = Vector{Float64}()
    chiSet = Vector{Float64}()
    for L in LList
        N = L*2
        adj = PeriodicBoundaryCondition(2, L)        

        for T in TList
            Lattice = InitLattice(N)
            mStep = zeros(Float64, MCstep)

            for step in 1:MCstep
                Wolf(Lattice, adj, N, T, 1.0)
                mStep[step] = abs(sum(Lattice)/N)
            end
            
            m, chi = BootStrap(mStep, 100, 20, T, N)
            append!(mSet, m)
            append!(chiSet, chi)
            println("[L$(L), T$(T)]")
        end
        
    end
    
    return mSet, chiSet
end

IsingWolf (generic function with 1 method)

In [ ]:
LList = [400, 800, 1600, 3200, 6400]
TList = collect(range(start=0.8, stop=1.5, length=8))
JList = collect(range(start=0.3, stop=2.1, length=7))

IsingSwendsen(LList, TList, JList, 500, 10)
#@time mList, chiList = IsingWolf(LList, TList, 1000) 